In [2]:
from datetime import datetime
import json

current = datetime.utcnow()
current

datetime.datetime(2023, 11, 26, 12, 38, 3, 602300)

In [3]:
json.dumps(current)

TypeError: Object of type datetime is not JSON serializable

In [7]:
def format_iso(dt):
    return dt.strftime('%Y-%m-%dT%H:%M:%S')

format_iso(current)

'2023-11-26T12:38:03'

In [8]:
current.isoformat()

'2023-11-26T12:38:03.602300'

In [9]:
log_record = {'time': datetime.utcnow().isoformat(),
              'message': 'testing'}

json.dumps(log_record)

'{"time": "2023-11-26T12:46:05.034513", "message": "testing"}'

In [14]:
log_record = {'time': datetime.utcnow(),
              'message': 'testing'}

json.dumps(log_record, default=format_iso)

'{"time": "2023-11-26T12:48:39", "message": "testing"}'

In [15]:
def format_general(arg):
    return "Unknown serialization"

json.dumps(log_record, default=format_general)

'{"time": "Unknown serialization", "message": "testing"}'

In [17]:
log_record = {'time': datetime.utcnow(),
              'message': 'testing',
              'args': {10, "test"}}

json.dumps(log_record, default=format_general)

'{"time": "Unknown serialization", "message": "testing", "args": "Unknown serialization"}'

In [24]:
log_record = {"time1": datetime.utcnow(),
              "time2": datetime.utcnow(),
              "message": "testing",
              "args": {10, "test"}
              }

json.dumps(log_record, default=format_iso)

AttributeError: 'set' object has no attribute 'strftime'

In [25]:
log_record = {"time1": datetime.utcnow(),
              "time2": datetime.utcnow(),
              "message": "testing",
              "args": {10, "test"}
              }

def custom_json_formatter(arg):
    if isinstance(arg, datetime):
        return arg.isoformat()
    elif isinstance(arg, set):
        return list(arg)
    
json.dumps(log_record, default=custom_json_formatter)

'{"time1": "2023-11-26T12:58:14.493240", "time2": "2023-11-26T12:58:14.493240", "message": "testing", "args": [10, "test"]}'

In [26]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        self.create_dt = datetime.utcnow()
    
    def __repr__(self):
        return f'Person(name={self.name}, age={self.age})'
    
    def toJSON(self):
        return {
            'name': self.name,
            'age': self.age,
            'create_dt': self.create_dt.isoformat()
        }
    
p = Person('John', 82)

print(p)

Person(name=John, age=82)


In [27]:
print(json.dumps(p, default=lambda obj: obj.toJSON()))

{"name": "John", "age": 82, "create_dt": "2023-11-26T13:00:14.844360"}


In [28]:
log_record = dict(time=datetime.utcnow(),
                  message="Created new person",
                  person=p)

json.dumps(log_record, default=custom_json_formatter)

'{"time": "2023-11-26T13:02:03.253747", "message": "Created new person", "person": null}'

In [29]:
def custom_json_formatter(arg):
    if isinstance(arg, datetime):
        return arg.isoformat()
    elif isinstance(arg, set):
        return list(arg)
    elif isinstance(arg, Person):
        return arg.toJSON()
    
log_record = dict(time=datetime.utcnow(),
                  message="Created new person",
                  person=p)

json.dumps(log_record, default=custom_json_formatter)

'{"time": "2023-11-26T13:03:12.229692", "message": "Created new person", "person": {"name": "John", "age": 82, "create_dt": "2023-11-26T13:00:14.844360"}}'

In [30]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        self.create_dt = datetime.utcnow()
    
    def __repr__(self):
        return f'Person(name={self.name}, age={self.age})'
    
    def toJSON(self):
        return {
            'name': self.name,
            'age': self.age,
            'create_dt': self.create_dt
        }
    

p = Person("Python", 45)

p

Person(name=Python, age=45)

In [31]:
log_record = dict(time=datetime.utcnow(),
                  message="Created new person",
                  person=p)

log_record

{'time': datetime.datetime(2023, 11, 26, 13, 5, 19, 959905),
 'message': 'Created new person',
 'person': Person(name=Python, age=45)}

In [32]:
json.dumps(log_record, default=custom_json_formatter)

'{"time": "2023-11-26T13:05:19.959905", "message": "Created new person", "person": {"name": "Python", "age": 45, "create_dt": "2023-11-26T13:04:57.325904"}}'

In [33]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        self.create_dt = datetime.utcnow()
    
    def __repr__(self):
        return f'Person(name={self.name}, age={self.age})'
    
    def toJSON(self):
        return vars(self)
    

p = Person("Python", 45)

p

Person(name=Python, age=45)

In [34]:
log_record = dict(time=datetime.utcnow(),
                  message="Created new person",
                  person=p)

log_record

{'time': datetime.datetime(2023, 11, 26, 13, 6, 40, 693338),
 'message': 'Created new person',
 'person': Person(name=Python, age=45)}

In [35]:
json.dumps(log_record, default=custom_json_formatter)

'{"time": "2023-11-26T13:06:40.693338", "message": "Created new person", "person": {"name": "Python", "age": 45, "create_dt": "2023-11-26T13:06:33.691668"}}'

In [37]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):
        return f'Point(x={self.x}, y={self.y})'

from decimal import Decimal

pt1 = Point(1, 2)
pt2 = Point(Decimal(10.5), Decimal(20.5))

p = Person("John", 82)

def custom_json_formatter(arg):
    if isinstance(arg, datetime):
        return arg.isoformat()
    elif isinstance(arg, set):
        return list(arg)
    
    else: 
        try:
            return arg.toJSON()
        except AttributeError:
            try:
                return vars(arg)
            except TypeError:
                return str(arg)
            
log_record = dict(time=datetime.utcnow(),
                  message="Created new point",
                  point=pt1, 
                  point2=pt2,
                  created_by=p)

In [39]:
json.dumps(log_record, default=custom_json_formatter)

'{"time": "2023-11-26T13:19:18.629820", "message": "Created new point", "point": {"x": 1, "y": 2}, "point2": {"x": "10.5", "y": "20.5"}, "created_by": {"name": "John", "age": 82, "create_dt": "2023-11-26T13:19:18.629820"}}'

In [42]:
from functools import singledispatch

@singledispatch
def json_format(arg):
    print(arg)
    try:
        print("\ttrying to use toJSON...")
        return arg.toJSON()
    except AttributeError:
        print("\tfailed - trying to use vars...")
        try:
            return vars(arg)
        except TypeError:
            print("\tfailed - using str...")
            return str(arg)
        

@json_format.register(datetime)
def _(arg):
    print(f"\tformatting datetime using isoformat...")
    return arg.isoformat()

@json_format.register(set)
def _(arg):
    print(f"\tformatting set using list...")
    return list(arg)


log_record = dict(time=datetime.utcnow(),
                  message="Created new point",
                  point=pt1, 
                  point2=pt2,
                  created_by=p)

print(json.dumps(log_record, default=json_format, indent=2))
    

	formatting datetime using isoformat...
Point(x=1, y=2)
	trying to use toJSON...
	failed - trying to use vars...
Point(x=10.5, y=20.5)
	trying to use toJSON...
	failed - trying to use vars...
10.5
	trying to use toJSON...
	failed - trying to use vars...
	failed - using str...
20.5
	trying to use toJSON...
	failed - trying to use vars...
	failed - using str...
Person(name=John, age=82)
	trying to use toJSON...
	formatting datetime using isoformat...
{
  "time": "2023-11-26T13:27:49.224388",
  "message": "Created new point",
  "point": {
    "x": 1,
    "y": 2
  },
  "point2": {
    "x": "10.5",
    "y": "20.5"
  },
  "created_by": {
    "name": "John",
    "age": 82,
    "create_dt": "2023-11-26T13:19:18.629820"
  }
}


In [45]:
from decimal import Decimal
from fractions import Fraction

d = dict(a=1+1j, b=Decimal('10.5'), c=Fraction(2, 3), p=Person('John', 82), pt=Point(1, 2), time=datetime.utcnow())

print(json.dumps(d, default=json_format, indent=2))

(1+1j)
	trying to use toJSON...
	failed - trying to use vars...
	failed - using str...
10.5
	trying to use toJSON...
	failed - trying to use vars...
	failed - using str...
2/3
	trying to use toJSON...
	failed - trying to use vars...
	failed - using str...
Person(name=John, age=82)
	trying to use toJSON...
	formatting datetime using isoformat...
Point(x=1, y=2)
	trying to use toJSON...
	failed - trying to use vars...
	formatting datetime using isoformat...
{
  "a": "(1+1j)",
  "b": "10.5",
  "c": "2/3",
  "p": {
    "name": "John",
    "age": 82,
    "create_dt": "2023-11-26T13:29:46.305452"
  },
  "pt": {
    "x": 1,
    "y": 2
  },
  "time": "2023-11-26T13:29:46.305452"
}


In [46]:
@json_format.register(Decimal)
def _(arg):
    print(f"\tformatting Decimal using float...")
    return f"Decimal({str(arg)})" 

In [47]:
print(json.dumps(d, default=json_format, indent=2))

(1+1j)
	trying to use toJSON...
	failed - trying to use vars...
	failed - using str...
	formatting Decimal using float...
2/3
	trying to use toJSON...
	failed - trying to use vars...
	failed - using str...
Person(name=John, age=82)
	trying to use toJSON...
	formatting datetime using isoformat...
Point(x=1, y=2)
	trying to use toJSON...
	failed - trying to use vars...
	formatting datetime using isoformat...
{
  "a": "(1+1j)",
  "b": "Decimal(10.5)",
  "c": "2/3",
  "p": {
    "name": "John",
    "age": 82,
    "create_dt": "2023-11-26T13:29:46.305452"
  },
  "pt": {
    "x": 1,
    "y": 2
  },
  "time": "2023-11-26T13:29:46.305452"
}
